# Cài đặt thư viện

## Cài đặt các thư viện chung

In [ ]:
import time
import os
import copy
import functools
import pandas as pd
import numpy as np

# Thư viện dựng biểu đồ và hình ảnh
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import OrderedDict

from tqdm import tqdm


## Cài đặt thư viện torch và torchvision

In [ ]:
import cv2
import torch
import torchvision


from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

#Albumentations là công cụ hữu ích trong thị giác máy tính giúp tăng hiệu suất xử lý cho mạng CNN
import albumentations as AL
from albumentations.pytorch import ToTensorV2

# Tải dữ liệu

In [ ]:
dataset = "../input/plant-pathology-2020-fgvc7"
train = pd.read_csv(os.path.join(dataset,"train.csv"))
test = pd.read_csv(os.path.join(dataset,"test.csv"))
print(f"Số lượng dữ liệu huấn luyện: {train.shape[0]}")
print(f"Số lượng dữ liệu dự đoán: {test.shape[0]}")

# Minh họa dữ liệu

## Phân nhóm các loại bệnh với số lượng tương ứng

In [ ]:
# Dùng hàm agg của pandas để tích hợp số lượng cho từng loại bệnh
quantities = train[train.columns[1:5]].agg(['sum']).loc['sum']

## Minh họa dữ liệu phân nhóm bằng Pie Chart

In [ ]:
# Tạo figure với kích thước và tỷ lệ
fig, ax = plt.subplots(figsize=(10, 5), subplot_kw=dict(aspect="equal"))

# Mảng các số lượng của từng loại bệnh
numbers = quantities.tolist()

# Mảng các tên của loại bệnh
types = quantities.keys().tolist()

# Hàm format nhãn khi xuất hiện trên chart (% và số lượng)
def formatting(v, data):
    absolute = int(round(v/100.*np.sum(data)))
    return "{:.1f}%\n{:d}".format(v, absolute)

# Tạo pie chart 
wedges, texts, autotexts = ax.pie(numbers, autopct=lambda v: formatting(v, numbers),
                                  textprops=dict(color="w"))

# Đây là 1 labelled pie chart nên phải có legend với các số liệu cấu hình cấu hình
ax.legend(wedges, types,
          title="Types",
          loc="center left",
          fontsize=12,
          bbox_to_anchor=(1, 0, 0.5, 1)).get_title().set_fontsize('12')

# Cài đặt cấu hình cho chữ trên pie chart
plt.setp(autotexts, size=12, weight="bold")

ax.set_title("Number of diseases by type pie chart", size=8, loc='center')

plt.show()

## Minh họa dữ liệu từ tập ảnh

In [ ]:
images = os.path.join(dataset,"images")

number_per_type = 3

# hàm chuyển đổi của reduce, bao gồm giá trí hiện tại và bộ chứa
def get_diseased_by_type(accum, cur):
    # lấy các hình ảnh có bệnh tương ứng với cột
    mask = train[cur] == 1
    
    # lấy 4 indexes của các hình ảnh có bệnh tương ứng với cột
    indexes = train[cur][mask].sample(n=number_per_type).keys()
    
    # lấy tên hình tương ứng với index
    images = [i + ".jpg" for i in train['image_id'][indexes]]
    
    accum.append(images)
    
    return accum

# lấy danh sách các bệnh
types = train.columns[1:5].tolist()

# Tương ứng với mỗi bệnh sẽ lấy {number_per_type} hình trong tập dữ liệu train
samples = functools.reduce(get_diseased_by_type, types, [])

fig = plt.figure(figsize=(20, 20))
i=1
for type_index in range(len(types)):
    samples_by_type = samples[type_index]
    for image_name in samples_by_type:
        ax = fig.add_subplot(len(types), number_per_type, i)
        img = mpimg.imread(os.path.join(images, image_name))
        imgplot = plt.imshow(img)
        ax.set_title(types[type_index])
        plt.axis('off')
        i+=1

# Xử lý và phân chia dữ liệu

## Tạo dữ liệu cross-validation từ tập train

In [ ]:
# Từ tên của hình ảnh để lấy ra đường dẫn tuyệt đối của ảnh trong thư mục input
def get_absolute_path(image_name):
    return os.path.join(dataset, "images", image_name + ".jpg")

train_images = train["image_id"].map(get_absolute_path)
train_labels = train[train.columns[1:5]]

# tách ra từ dữ liệu train để làm dữ liệu cross-validation với tỷ lệ (train, valid) = (80, 20)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size = 0.2, random_state=16)

# Mỗi tập dữ liệu sinh ra sau khi tách vẫn dữ nguyên index trong dataframe ban đầu nên ta cần reset index của nó (bắt đầu từ 0)
# drop=True để remove cột index cũ
train_images.reset_index(drop=True,inplace=True)
train_labels.reset_index(drop=True,inplace=True)
val_images.reset_index(drop=True,inplace=True)
val_labels.reset_index(drop=True,inplace=True)

## Định nghĩa các transform để xử lý dữ liệu ảnh theo các tùy chọn để tăng hiệu quả cho mô hình

In [ ]:
# Sử dụng các transforms của pytorch
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        
        # ngẫu nhiên crop một vị trí của hình và resize nó với 1 kích thước cho trước
        transforms.RandomResizedCrop(size=(256, 256)),
        
        # Lật hình theo hướng horizontal
        transforms.RandomHorizontalFlip(p=0.5),
        
        # Lật hình theo hướng vertical
        transforms.RandomVerticalFlip(p=0.5),
        
        # Rotate hình với ngẫu nhiên góc đó, vị trí và tỷ lệ phóng
        transforms.RandomAffine(degrees=(0, 180)),
        
        
        
        # giá trị trung bình mean=[0.485, 0.456, 0.406] (thường dùng)
        # giá trị độ lệch chuẩn st=[0.229, 0.224, 0.225] (thường dùng)
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomResizedCrop(size=(256, 256)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Sử dụng các transforms của Albumentations
# mytransform = {
#     "train": AL.Compose([
#     AL.RandomResizedCrop(height=256, width=256, p=1.0),
#     AL.Flip(),
#     AL.ShiftScaleRotate(rotate_limit=1.0, p=0.8),
#     AL.Normalize(p=1.0),
#     ToTensorV2(p=1.0),
#     ]),
#     "validation": AL.Compose([
#     AL.RandomResizedCrop(height=256, width=256, p=1.0),
#     AL.Normalize(p=1.0),
#     ToTensorV2(p=1.0),
#     ]),
# }


## Class ImageDataset để thực hiện xử lý dữ liệu với các transform

In [ ]:
class ImageDataset(Dataset):
    
    def __init__(self, images, labels=None, transforms=None):
        self.images = images
        self.labels = labels
        self.transforms=transforms
        
    def __getitem__(self, idx):
        # đọc hình ảnh với vị trí idx trong tập ảnh
        image = cv2.imread(self.images[idx])
        
        # chuyển đổi ảnh sang hệ số màu RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # transform hình ảnh
        transformed = image if transforms is None else self.transforms(image)
        
        if self.labels is None:
            return transformed
        else:
            label = None if self.labels is None else torch.argmax(torch.tensor(self.labels.iloc[idx]))
            return transformed, label

    def __len__(self):
        return self.images.shape[0]

# Mô hình huấn luyện

## Các cấu hình

In [ ]:
BATCH_SIZE = 64 #4
NUM_EPOCHS = 15 #10
device = "cuda"
TRAIN_SIZE = train_labels.shape[0]
VALID_SIZE = val_labels.shape[0]
learning_rate = 5e-5

# pre-trained model sử dụng
model_name = "resnet"

# số loại bệnh
num_classes = 4

# chỉ tối ưu các trọng số của fined-tune network
feature_extract = True

## Chọn mô hình pre-trained

In [ ]:
# Liệu rằng chúng ta có nên tối ưu trọng số của cả mô hình gồm pre-train và fined-tune network,
# Nếu feature_extracting = True, nghĩa là chỉ cập nhật fined-tune network
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Khởi tạo mô hình mạng sử dụng
def initialize_model(model_name, num_classes, feature_extract=True, use_pretrained=True):
    model_ft = None

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Bởi vì lớp fully connected cuối cùng của Resnet18 có dạng Linear(in_features=512, out_features=1000, bias=True)
        # nên chúng ta phải thay đổi lớp cuối cùng này với {out_feature} bằng {num_classes}
        in_features = model_ft.fc.in_features
        # Lớp fined-tune, sẽ được cập nhật trong số trong khi train
        model_ft.fc = nn.Linear(in_features, num_classes)

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft


## Tạo mô hình pre-trained

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model for this run
model_ft = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

## Tạo trình tối ưu (optimizer)

In [ ]:
# nạp mô hình vào GPU
model_ft = model_ft.to(device)

# feature_extract == True, nghĩa là chỉ lấy những parameters cần cập nhật, ngược lại thì sẽ cập nhật trọng số của cả network
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    # lấy tất cả param của mạng
    params_to_update = model_ft.parameters()
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


# Sử dụng stomach GD optimizer
optimizer_ft = optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)


## Tạo CrossEntropyLoss

In [ ]:
criterion = nn.CrossEntropyLoss()

## Hàm huấn luyện

In [ ]:
def train(model, dataloaders, criterion, optimizer, num_epochs=15):
    since = time.time()
    
    train_loss_history = []
    val_loss_history = []
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Mỗi epoch sẽ chạy trên phase train và phase validation
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # load dữ liệu từ dataloader theo batch_size
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # reset độ dốc để tránh cộng dồn
                optimizer.zero_grad()

                # Chỉ cập nhật trọng số trong quá trình train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # Lấy vị trí của giá trị cao nhất
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        
                        # lan truyền giá trị lỗi đến các trọng số trong mạng
                        loss.backward()
                        
                        # cập nhật lại các trọng số
                        optimizer.step()

                # Cộng dồn giá trị lỗi trên mỗi batch
                running_loss += loss.item() * inputs.size(0)
                
                # Cộng dồn độ chính xác trên mỗi batch
                running_corrects += torch.sum(preds == labels.data)

            # Tính lỗi trung bình trên mỗi item trên mỗi phase, mỗi epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            
            # Tính độ chính xác trên mỗi phase, mỗi epoch
            epoch_acc = running_corrects.double().item() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
                train_loss_history.append(epoch_loss)
                
            if phase == 'val':
                val_loss_history.append(epoch_loss)
                val_acc_history.append(epoch_acc)
                
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            

        print(f"######## End epoch {epoch} ##########")

    time_elapsed = time.time() - since
    print('Thời gian huấn luyện: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Độ chính xác tốt nhất trên tập validation: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_loss_history, val_loss_history, val_acc_history





# Huấn luyện mô hình

## Nạp dữ liệu vào Pytorch Dataloader

In [ ]:
train_data_loader = DataLoader(ImageDataset(train_images, train_labels, data_transforms["train"]), batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(ImageDataset(val_images, val_labels, data_transforms["val"]), batch_size=BATCH_SIZE, shuffle=True)

dataloaders_dict = {"train": train_data_loader, "val": val_data_loader}

## Thực thi huấn luyện

In [ ]:
# Train and evaluate
model_ft, train_loss, val_loss, val_acc = train(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=NUM_EPOCHS)

# Minh họa lỗi

In [ ]:
metrics = [{"label": "Training Loss", "value": train_loss},
           {"label": "Validation Loss", "value": val_loss},
           {"label": "Validation Accuracy", "value": val_acc}]
fig = plt.figure(figsize=(20, 5))

for i, m in enumerate(metrics):
    fig.add_subplot(1, 3, i+1)
    plt.title(f"{m['label']} vs. Number of Training Epochs")
    plt.xlabel("Training Epochs")
    plt.ylabel(f"{m['label']}")
    plt.plot(range(1,NUM_EPOCHS+1), m['value'])

plt.show()
   
    

# Dự đoán

In [ ]:
softmax = nn.Softmax(dim=1)

def test_function(model, loader):
    predictions = []
    
    progress = tqdm(loader, desc="Testing")
    #print(progress)
    #inputs, labels
    with torch.no_grad():
        for  _, images in enumerate(progress):
            images = images.to(device)
            model.eval()
            probs = softmax(model(images)).tolist()
            predictions.extend(probs)
    return predictions

test_images = test["image_id"].map(get_absolute_path)
test_loader = DataLoader(ImageDataset(test_images, None, data_transforms["val"]), batch_size=BATCH_SIZE)
predictions = np.array(test_function(model_ft, test_loader))
print(predictions)


# Submit

In [ ]:
submission = pd.read_csv(os.path.join(dataset,"sample_submission.csv"))
submission.loc[:, submission.columns[1:5]] = predictions
submission.to_csv("submission_2.csv", index=False)